# SIM Pattern Generator

### The Python version

From the [SI of the paper](http://www.pnas.org/content/suppl/2012/03/15/1119262109.DCSupplemental/pnas.201119262SI.pdf):



In [1]:
%pylab inline
# nice plotting
# import seaborn as sns
import numexpr as ne
# for minimizing the difference between the desired frequency and the calculated one
from scipy.optimize import minimize
# Need to be able to do polynomial fitting
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
# PIL allows us to write binary images, though we need a cludge, see 'Writing Binary Files.ipynb'
from PIL import Image
import os
import zipfile
from pyfftw.interfaces.numpy_fft import fftn
import pyfftw
pyfftw.interfaces.cache.enable()
from scipy.ndimage import gaussian_filter

from skimage.draw import circle

from matplotlib.colors import LogNorm

Populating the interactive namespace from numpy and matplotlib


In [2]:
# newest one
def pattern_gen(angle, period, onfrac = 0.5, phase_idx = 0., phase_offset = 0., nphases =5,
                sizex =2048, sizey =1536, SIM_2D = True):
    '''
    Generates a binary SLM pattern for SIM
    
    Generates a sine wave and then binarizes it
    
    designed for my orientation
    
    Parameters
    ----------
    x : array_like
        the $\vec{a}$ from [1]. Defines the pattern orientation
    period : float
        Defines the period of the pattern
    onfrac : float
        The fraction of on pixels in the pattern
    phase_idx : int
        The phase of the pattern (see `nphases` for more info)
    phase_offset : float
        The offset in phase, mostly used for aligning patterns of different colors
    nphases : int
        the number of phases
    sizex : int
        size of the pattern
    sizey : int
        size of the pattern

    Returns
    -------
    pattern : ndarray
        A binary array representing a single bitplane to send to the SLM
    '''
    
    if not 0 < onfrac < 1:
        raise ValueError('onfrac must have a value between 0 and 1. onfrac = {}'.format(onfrac))
    
    
    if SIM_2D:
        # Then we only want to take steps of 2pi/n in illumination which means pi/n at the SLM
        phase_step = 2
    else:
        phase_step = 1
    
    # indices are backwards here for backwards compatability with older generation code.
    yy, xx = meshgrid(arange(sizey),arange(sizex),indexing='ij')
    
    # here's the pattern frequency
    freq = 2*pi/period
    
    phi = (phase_idx/nphases/phase_step)*(2*pi) + phase_offset
    
    # our sine goes from -1 to 1 while onfrac goes from 0,1 so move onfrac into the right range
    onfrac = onfrac*2-1
    
    # given angle is the angle of the fourier spots from the 'i' vector, so pattern should be 90 degrees from this
    a = angle
    
    # do the evaluation
    toreturn = ne.evaluate("sin(freq*(cos(a)*xx+sin(a)*yy)+phi) > onfrac")
    
    return toreturn

In [3]:
def angles(init_angle, num_angles):
    thetas = arange(0., num_angles)*pi/1./num_angles + init_angle
    return thetas

In [4]:
def ideal_period(wavelength, NA = 0.85):
    '''
    All units are in mm
    '''
    pixel_size = 8.2/1000 # pixel size in mm for QXGA display (4DD)
    fl = 250 # focal length of lens in mm
    fl2 = 300 # focal length of the second lens
    ftube = 200 # focal length of the tube lens, for Nikon this is 200 mm
    wl = wavelength/10**6 # wavelength of light
    mag = 1/100
    sigma = sqrt(2) * 12/pixel_size/4 # std dev of gaussian beam in units of pixels at the SLM
    pupil_diameter = 2*NA*mag*fl2    # Size of pupil image at first fourier plane
    hole_radius = 2*wl*fl/(2* pi * sigma *sqrt(2) * pixel_size) # this is the limit of hole size
    hole_radius = 0.1/2# this is more reasonable (50 um)
    period = wl * fl * (1/(pupil_diameter/2 - hole_radius))/ pixel_size # in mm
    
    return period

In [5]:
def pattern_params(angle, period, onfrac = 0.5, phaseInd = 0., phaseOffset = 0., nphases = 5, sizex =2048, sizey =1536):
    '''
    Find the precise pattern period
    
    Using 2nd order polynomial fit along either axis
    '''
    
    # this is a standin for nextpow2, I don't know if there's a more efficient one, but its not going to
    # be a bottle neck
    n = 1<<(min(sizex,sizey)-1).bit_length()
    
    my_pat = pattern_gen(angle, period, onfrac, phaseInd, phaseOffset, nphases, n, n)
    
    my_pat_fft = abs(ifftshift(fftn(fftshift(my_pat))))

    dc_loc = ((np.array(my_pat_fft.shape)+1)//2)
    
    # make mask
    mask = ones_like(my_pat_fft)
    mask[circle(*dc_loc,radius=10)]=0
    mask[:dc_loc[0]]=0
    
    # mask data and find next biggest peak
    masked_fft_data = abs(mask*my_pat_fft)
    max_loc = unravel_index(masked_fft_data.argmax(),masked_fft_data.shape)
    # pull the 3x3 region around the peak
    region_size = 3
    start = -(region_size-1)//2
    end = region_size+start

    my_pat_fft_suby = masked_fft_data[max_loc[0]+start:max_loc[0]+end,max_loc[1]]
    my_pat_fft_subx = masked_fft_data[max_loc[0],max_loc[1]+start:max_loc[1]+end]

    x = arange(start,end)
    
    xfit = polyfit(x, my_pat_fft_subx,2)
    yfit = polyfit(x, my_pat_fft_suby,2)

    x0 = -xfit[1]/(2*xfit[0])
    y0 = -yfit[1]/(2*yfit[0])

    precisepeak = array(max_loc)+array([x0,y0])-array(dc_loc)

    preciseangle = arctan2(precisepeak[0],precisepeak[1])
    
    if angle < 0:
        preciseangle -= pi
    
    precise_period = n/norm(precisepeak)
    
    return {"period" : precise_period, "angle" : preciseangle, "pat" : my_pat, "pat fft" : my_pat_fft}

In [6]:
def opt_period(iperiod,angle,**kwargs):
    def objf_l1(period):
        data = array([pattern_params(angle, period, **kwargs)['period'] for n in range(1)])
        return mean(abs(data-iperiod))
    
    return minimize(objf_l1, iperiod ,method='Nelder-Mead')['x']

In [7]:
def opt_angle(period,iangle,**kwargs):
    def objf_l1(angle):
        data = array([pattern_params(angle, period, **kwargs)['angle'] for n in range(1)])
        return mean(abs(data-iangle))
    
    return minimize(objf_l1, iperiod ,method='Nelder-Mead')['x']

In [8]:
'''
We can use sets as a way to keep track of images and sequences.

Images and sequences will be denoted by strings referring to the actual file name

A repetoire will only contain a list of RunningOrders and have a method to write out running orders.

It should also have the ability to read in a .rep file

Need a method to return a given running order, by name
'''

class RunningOrder(object):
    '''
    A class representing a running order for a 4DD SLM
    '''
    
    def __init__(self, name, angles=3, periods=1, triggered = True):
        self._name = name
        self._periods = periods
        self._angles = angles
        # we want these properties to be immutable and we don't want duplicates
        self._sequences = set()
        self._bitplanes = set()
        self.triggered = triggered
        self._RO = []
        
    @property
    def name(self):
        return self._name
    
    @property
    def periods(self):
        return self._periods
    
    @property
    def angles(self):
        return self._angles
    
    @property
    def sequences(self):
        return self._sequences
    
    @property
    def bitplanes(self):
        return self._bitplanes
    
    @property
    def RO(self):
        return self._RO
    
    def addbitplane(self, sequence, bitplane, position = None):
        '''
        A function to add sequence bitplane pairs to a running order
        '''
        self._sequences.add(sequence)
        self._bitplanes.add(bitplane)
            
        seq_bp = {'sequence' : sequence, 'bitplane' : bitplane}
        
        if position is None:
            self._RO.append(seq_bp)
        else:
            self._RO.insert(position, seq_bp)

class Repetoire(object):
    '''
    This object holds a list of Running orders. It has the ability to output .rep files
    
    Further functionality will be to read in .rep files.
    '''
    
    def __init__(self, listofROs = None):
        self._ROs = []
        self._sequences = set()
        self._bitplanes = set()
        if listofROs is not None:
            for RO in listofROs:
                self.add_RO(RO)
        
    def add_RO(self, RO):
        
        if isinstance(RO, list):
            self.add_RO(RO.pop())
        
        if not isinstance(RO, RunningOrder):
            raise TypeError('{} is not a RunningOrder'.format(RO))
        else:
            self._ROs.append(RO)
            self._sequences |= RO.sequences
            self._bitplanes |= RO.bitplanes

    def __str__(self):
        '''
        This function writes the repetoire
        '''
        
        toreturn = ''
        
        # If ROs is empty
        if not len(self._ROs):
            return toreturn
        
        # # # # # # # # # # # # # # # # # # # # 
        # MAKING SEQUENCES # 
        # # # # # # # # # # # # # # # # # # # # 
        
        # need to save the letter assignments
        seq_dict = {}
        
        seqs = ['SEQUENCES']
        for i, seq in enumerate(sorted(self._sequences)):
            char = chr(65+i)
            seqs.append(char+' "'+seq+'"')
            seq_dict[seq] = char
        
        seqs.append('SEQUENCES_END\n\n')
        
        toreturn = "\n".join(seqs)
        
        # # # # # # # # # # # # # # # # # # # # 
        # MAKING BITPLANES # 
        # # # # # # # # # # # # # # # # # # # # 
        
        bp_dict = {}
        bps = ['IMAGES']
        for i, bp in enumerate(sorted(self._bitplanes)):
            bps.append('1 "'+bp+'"')
            bp_dict[bp] = i
        
        bps.append('IMAGES_END\n\n')
        
        toreturn += "\n".join(bps)
        
        # Need to have at least one RunningOrder be Default, why not the first one?
        # If no DEFAULT then MetroCon will continue to look for one and fail.
        toreturn += 'DEFAULT '
        # sort the RunningOrders by name
        for RO in self._ROs:# .sort(key=lambda x: x.name):
            toreturn += self._writeRO(RO, seq_dict, bp_dict)
            
        return toreturn
        
    @staticmethod
    def _writeRO(RO, seq_dict, bp_dict):
        '''
        This function will write the repetoire based on the sequence dict and image dict
        
        It assumes that all that is wanted is standard SIM sequences
        '''
        
        toreturn = ['"{}"'.format(RO.name)]
        
        toreturn.append('[HWA\n')
        

        if not RO.triggered:
            toappend = ' {'
            
        try:
            a = array(RO.RO).reshape(RO.angles,-1)
        except ValueError:
            raise ValueError('{} has length = {} which is not divisible by {}'.format(RO.name, len(RO.RO), RO.angles))
        else:
            for angle in array(RO.RO).reshape(RO.angles,-1):
                for i in range(RO.periods):
                    for seq_bp in angle:
                        seq = seq_bp['sequence']
                        bp = seq_bp['bitplane']
                        if RO.triggered:
                            toreturn.append(' <t({0},{1}) >\n {{f ({0},{1}) }}'.format(seq_dict[seq],bp_dict[bp]))
                        else:
                            toappend += '({0},{1}) '.format(seq_dict[seq],bp_dict[bp])

        if not RO.triggered:
            toappend += '}'
            toreturn.append(toappend)

        toreturn.append(']\n\n')
        
        return '\n'.join(toreturn)
        

In [20]:
# Super SIM for 5 orientations
onfrac = 0.5
num_angles = 5
my_angles = angles(deg2rad(9.0),num_angles)
    
num_phases = 15
num_periods = 1

wls = (561,488)

repname = 'SuperSIM488_561_2D_5orients_9deg'
# open zip file, make sure the compression is set to ZIP_DEFLATED (8)
with zipfile.ZipFile('{}.repz11'.format(repname), 'w', compression=zipfile.ZIP_DEFLATED) as myzip:

    opt = False

    my_dir = os.path.join('{}'.format(repname),'')
    if not os.path.isdir(my_dir):
        print('Making ' + my_dir)
        os.mkdir(my_dir)
    
    NAs = (0.81, 0.82, 0.83, 0.84)
    
    ROlist = []
    
    seq = '48070 HHMI 10ms.seq11'
    seq2 = '48077 HHMI 24 50ms.seq11'
    
    for myseq in (seq,):
        myzip.write(os.path.join('HHMI_R11_Seq', myseq),arcname=myseq)
    
    for wl in wls:
        for NA in NAs:
            print('{} nm, NA {}'.format(wl,NA))

            iperiod = ideal_period(wl,NA=NA)
            # ROs here
            super_RO = RunningOrder('{} nm Super SIM {}pi NA {}'.format(wl, num_periods*2, NA),
                                    angles=num_angles, periods=num_periods)
            reg_RO_3 = RunningOrder('{} nm SIM NA {} 3 phases'.format(wl,NA), angles=num_angles)
            reg_RO_5 = RunningOrder('{} nm SIM NA {} 5 phases'.format(wl,NA), angles=num_angles)

            RO_all_t = RunningOrder('{} nm NA {} All Orientations Triggered'.format(wl,NA), angles=1)
            RO_all = RunningOrder('{} nm NA {} All Orientations'.format(wl,NA), triggered=False, angles=1)

            patall =zeros((1536, 2048))

            for angle in my_angles:

                # calculate degree from angle for filename
                degree = rad2deg(angle)

                RO1orient = RunningOrder('{} nm NA {} {:+.2f} degrees'.format(wl, NA, degree), angles=1, triggered=False)

                # optimize the angle
                if opt:
                    print('Optimizing angle = {:+.2f} degrees'.format(degree))
                    my_per = opt_period(iperiod,angle)[0]
                else:
                    my_per = iperiod

                for phase in range(num_phases):
                    # generate pattern, with one period,
                    pat = pattern_gen(angle, my_per,onfrac=onfrac,phase_idx=phase,nphases=num_phases,SIM_2D=True)

                    # make image object
                    pat_img = Image.fromarray((pat*255).astype('uint8'),mode='L')

                    # generate filename, put orientation in front, so that its consistent
                    name = 'pat-{}nm-{:.2f}NA{:+.1f}deg-{:02d}ph-{:.4f}pix-{:.2f}DC.bmp'.format(wl,NA, degree,phase,my_per,onfrac)

                    # convert to binary and save
                    pat_img.convert('1').save(os.path.join(my_dir,name))

                    # move to zip file and save
                    myzip.write(os.path.join(my_dir,name),arcname=name)

                    super_RO.addbitplane(seq,name)

                    if not phase:
                        RO1orient.addbitplane(seq,name)
                        # add to all angles pattern
                        patall += pat

                    # only pull the first three phases so that we can do regular SIM too
                    if (phase % (num_phases//3) == 0):
                        # num_phases//desired number of phases will calculate the right divisor
                        # check
                        assert num_phases % 3 == 0
                        reg_RO_3.addbitplane(seq,name)
                    if (phase % (num_phases//5) == 0):
                        assert num_phases % 5 == 0
                        reg_RO_5.addbitplane(seq,name)

                    print('Wrote file: {}'.format(name))

                ROlist.append(RO1orient)

            # make image object for all
            patall_img = Image.fromarray((patall/num_angles*255).astype('uint8'),mode='L')

            # generate filename, put orientation in front, so that its consistent
            nameall = 'pat-{}nm-{:.2f}NA-{:.4f}pix-{:.2f}DC_allangles.bmp'.format(wl,NA,my_per,onfrac)

            # convert to binary and save
            patall_img.convert('1').save(os.path.join(my_dir,nameall))

            # move to zip file and save
            myzip.write(os.path.join(my_dir,nameall),arcname=nameall)

            RO_all_t.addbitplane(seq,nameall)
            RO_all.addbitplane(seq,nameall)

            ROlist.append(super_RO)
            ROlist.append(reg_RO_3)
            ROlist.append(reg_RO_5)
            ROlist.append(RO_all_t)
            ROlist.append(RO_all)
                
    my_Rep = Repetoire(ROlist)
    
    # write the rep file
    with open(os.path.join(my_dir,'{}.rep'.format(repname)),'w') as repfile:
        repfile.write(str(my_Rep))
        
    # now add the file to the zip
    myzip.write(os.path.join(my_dir,'{}.rep'.format(repname)),arcname='{}.rep'.format(repname))
    
print(my_Rep)

561 nm, NA 0.81
Wrote file: pat-561nm-0.81NA+9.0deg-00ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-01ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-02ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-03ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-04ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-05ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-06ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-07ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-08ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-09ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-10ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-11ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-12ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-13ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+9.0deg-14ph-7.1864pix-0.50DC.bmp
Wrote file: pat-561nm-0.81NA+45.0deg-00ph-7.1864pix-0.

In [19]:
# Super SIM for 5 orientations
onfrac = 0.5
num_angles = 5
my_angles = angles(deg2rad(9.0),num_angles)
    
num_phases = 15
num_periods = 1

wls = (488,)

repname = 'SuperSIM488_2D_5orients_9deg'
# open zip file, make sure the compression is set to ZIP_DEFLATED (8)
with zipfile.ZipFile('{}.repz11'.format(repname), 'w', compression=zipfile.ZIP_DEFLATED) as myzip:

    opt = False

    my_dir = os.path.join('{}'.format(repname),'')
    if not os.path.isdir(my_dir):
        print('Making '+my_dir)
        os.mkdir(my_dir)
    
    NAs = (0.81, 0.82) #, 0.83, 0.84)
    
    ROlist = []
    
    seq = '48070 HHMI 10ms.seq11'
    
    for myseq in (seq,):
        myzip.write(os.path.join('HHMI_R11_Seq', myseq),arcname=myseq)
    
    for wl in wls:
        for NA in NAs:
            print('{} nm, NA {}'.format(wl,NA))

            iperiod = ideal_period(wl,NA=NA)
            # ROs here
            super_RO = RunningOrder('{} nm Super SIM {}pi NA {}'.format(wl, num_periods*2, NA),
                                    angles=num_angles, periods=num_periods)
            reg_RO_3 = RunningOrder('{} nm SIM NA {} 3 phases'.format(wl,NA), angles=num_angles)
            reg_RO_5 = RunningOrder('{} nm SIM NA {} 5 phases'.format(wl,NA), angles=num_angles)

            RO_all_t = RunningOrder('{} nm NA {} All Orientations Triggered'.format(wl,NA), angles=1)
            RO_all = RunningOrder('{} nm NA {} All Orientations'.format(wl,NA), triggered=False, angles=1)

            patall =zeros((1536, 2048))

            for angle in my_angles:

                # calculate degree from angle for filename
                degree = rad2deg(angle)

                RO1orient = RunningOrder('{} nm NA {} {:+.2f} degrees'.format(wl, NA, degree), angles=1, triggered=False)

                # optimize the angle
                if opt:
                    print('Optimizing angle = {:+.2f} degrees'.format(degree))
                    my_per = opt_period(iperiod,angle)[0]
                else:
                    my_per = iperiod

                for phase in range(num_phases):
                    # generate pattern, with one period,
                    pat = pattern_gen(angle, my_per,onfrac=onfrac,phase_idx=phase,nphases=num_phases,SIM_2D=True)

                    # make image object
                    pat_img = Image.fromarray((pat*255).astype('uint8'),mode='L')

                    # generate filename, put orientation in front, so that its consistent
                    name = 'pat-{}nm-{:.2f}NA{:+.1f}deg-{:02d}ph-{:.4f}pix-{:.2f}DC.bmp'.format(wl,NA, degree,phase,my_per,onfrac)

                    # convert to binary and save
                    pat_img.convert('1').save(os.path.join(my_dir,name))

                    # move to zip file and save
                    myzip.write(os.path.join(my_dir,name),arcname=name)

                    super_RO.addbitplane(seq,name)

                    if not phase:
                        RO1orient.addbitplane(seq,name)
                        # add to all angles pattern
                        patall += pat

                    # only pull the first three phases so that we can do regular SIM too
                    if (phase % (num_phases//3) == 0):
                        # num_phases//desired number of phases will calculate the right divisor
                        # check
                        assert num_phases % 3 == 0
                        reg_RO_3.addbitplane(seq,name)
                    if (phase % (num_phases//5) == 0):
                        assert num_phases % 5 == 0
                        reg_RO_5.addbitplane(seq,name)

                    print('Wrote file: {}'.format(name))

                ROlist.append(RO1orient)

            # make image object for all
            patall_img = Image.fromarray((patall/num_angles*255).astype('uint8'),mode='L')

            # generate filename, put orientation in front, so that its consistent
            nameall = 'pat-{}nm-{:.2f}NA-{:.4f}pix-{:.2f}DC_allangles.bmp'.format(wl,NA,my_per,onfrac)

            # convert to binary and save
            patall_img.convert('1').save(os.path.join(my_dir,nameall))

            # move to zip file and save
            myzip.write(os.path.join(my_dir,nameall),arcname=nameall)

            RO_all_t.addbitplane(seq,nameall)
            RO_all.addbitplane(seq,nameall)

            ROlist.append(super_RO)
            ROlist.append(reg_RO_3)
            ROlist.append(reg_RO_5)
            ROlist.append(RO_all_t)
            ROlist.append(RO_all)
                
    my_Rep = Repetoire(ROlist)
    
    # write the rep file
    with open(os.path.join(my_dir,'{}.rep'.format(repname)),'w') as repfile:
        repfile.write(str(my_Rep))
        
    # now add the file to the zip
    myzip.write(os.path.join(my_dir,'{}.rep'.format(repname)),arcname='{}.rep'.format(repname))
    
print(my_Rep)

488 nm, NA 0.81
Wrote file: pat-488nm-0.81NA+9.0deg-00ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-01ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-02ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-03ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-04ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-05ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-06ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-07ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-08ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-09ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-10ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-11ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-12ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-13ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+9.0deg-14ph-6.2513pix-0.50DC.bmp
Wrote file: pat-488nm-0.81NA+45.0deg-00ph-6.2513pix-0.